In [ ]:
import matplotlib.pylab as plt
from sklearn.metrics import classification_report

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, GlobalMaxPooling2D
from keras.optimizers import Adam

import tensorflow as tf
from keras import backend as K
from keras import initializers

from tensorflow.keras.models import load_model, Model
from tensorflow.python.keras import backend as K
from tqdm import tqdm
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from keras.models import Sequential,Model,load_model
import time

import keras
import numpy as np
import random

In [ ]:
import os
import re
import cv2
import numpy as np

from multiprocessing import Process

W = 50
def load_photos_from_folders(path):

  images = []
  img = os.listdir(path)
  counter = 0
  for i in tqdm (range(len(img))):
    filenames = img[counter]
    # filename = re.split('_+' , filenames)

    images.append([])
    full_path = path+filenames

    image = cv2.imread(full_path)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (W, W)) 
    image = np.asarray(image)
    image = image.astype('float32')
    image /= 255.0
    
    images[counter].append(image)
    counter = counter+1
    # print(filename, "Loaded")
  return images

def load_Age_labels_from_folder(path): #Age
  filenames = os.listdir(path) 

  category = []
  counter = 0
  for filename in filenames:

      items = re.split('_+', filename)

      category.append([])
      age = re.split('A',items[0])

      age = int(age[1])
      if age <= 10:
        category[counter].append(0) 
      elif age > 10 and age <= 20 :
        category[counter].append(1)
      elif age > 20 and age <= 40 :
        category[counter].append(2) 
      else:
        category[counter].append(3)
      counter = counter+1
  return category


def load_Gender_labels_from_folder(path): #Gender
  filenames = os.listdir(path)

  category = []
  counter = 0
  for filename in filenames:
    items = re.split('_+', filename)
    gender = (items[1])
    category.append([])
    if gender.startswith("01"):
        category[counter].append(0) #male
    else:
      category[counter].append(1) #female
    counter = counter+1
  return category


def load_Expression_labels_from_folder(path): #Expression
  filenames = os.listdir(path)

  category = []
  counter = 0
  for filename in filenames:
    items = re.split('_+', filename)
    expression = re.split(' ',items[2])
    expression = expression[0]
    category.append([])
    if expression == "01":
      category[counter].append(0) #happy
    elif expression == "02":
      category[counter].append(1) #nutral
    else:
      category[counter].append(2) #not happy
    counter = counter+1
  return category

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

def make_x_y(images, age, gender, expression):

  X = [i for i in images]
  Y1 = [j for j in age]
  Y2 = [k for k in gender]
  Y3 = [k for k in expression]

  mlb = MultiLabelBinarizer()
  Y1 = mlb.fit_transform(Y1)
  Y2= mlb.fit_transform(Y2)
  Y3 = mlb.fit_transform(Y3)
  # print("Y1", Y1, "Y2",Y2, "Y3", Y3)
  X, Y1, Y2, Y3 = np.array(X), np.array(Y1), np.array(Y2), np.array(Y3)
  return X, Y1, Y2, Y3

def prepare_train_test(X, Y1, Y2, Y3, W, random_state = 42):
  X_train, X_test, Y_train, Y_test, Y_train2, Y_test2, Y_train3, Y_test3 = train_test_split(X, Y1, Y2, Y3, random_state = random_state)
    
  X_train, X_test, Y_train, Y_test, Y_train2, Y_test2, Y_train3, Y_test3 = np.array(X_train), np.array(X_test), np.array(Y_train), np.array(Y_test), np.array(Y_train2), np.array(Y_test2), np.array(Y_train3), np.array(Y_test3)

  X_train = X_train.reshape(X_train.shape[0], W, W, 3)
  X_test = X_test.reshape(X_test.shape[0], W, W, 3)

  return X_train, X_test, Y_train, Y_test, Y_train2, Y_test2, Y_train3, Y_test3

In [ ]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)
my_init = initializers.glorot_uniform(seed=42)

In [ ]:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

In [ ]:
images = load_photos_from_folders("./Dataset/")
age = load_Age_labels_from_folder("./Dataset/")
gender = load_Gender_labels_from_folder("./Dataset/")
expression = load_Expression_labels_from_folder("./Dataset/")

In [ ]:
X, Y1, Y2, Y3 = make_x_y(images, age, gender, expression)

In [ ]:
X_train, X_test, Y_train, Y_test, Y_train2, Y_test2, Y_train3, Y_test3 = prepare_train_test(X, Y1, Y2, Y3, W)

In [ ]:
input_shape = (W, W, 3)

**MULTITASKING**

In [ ]:
# rm -rf ./logs/

In [ ]:
visible = Input(shape=input_shape)
x1 = Conv2D(10, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(visible)
x1 = MaxPooling2D(pool_size=(2, 2))(x1)
x2 = Conv2D(20, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x1)
x2 = MaxPooling2D(pool_size=(2, 2))(x2)
x3 = Conv2D(30, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x2)
x = GlobalMaxPooling2D()(x3)
output = Dense(Y_train.shape[1], activation='softmax', name = 'age', kernel_initializer=my_init)(x)
output2 = Dense(Y_train2.shape[1], activation='sigmoid',name = 'gender', kernel_initializer=my_init)(GlobalMaxPooling2D()(x2))
output3 = Dense(Y_train3.shape[1], activation='softmax', name = 'expression', kernel_initializer=my_init)(GlobalMaxPooling2D()(x1))
model = Model(inputs=visible, outputs=[output, output2, output3])

In [ ]:
model.summary()

In [ ]:
NAME = 'multitask'
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

earlystop = EarlyStopping (monitor='val_loss', mode='min', verbose=1, patience=10)
callbacks = [tensorboard,earlystop]

In [ ]:
model.compile(loss=['categorical_crossentropy', 'binary_crossentropy', 'mse'],
              optimizer=Adam(clipnorm = 1.), metrics = {'age': 'accuracy', 'gender': 'accuracy', 'expression' : 'accuracy'}, loss_weights = [1, 1e-1, 1e-1])

In [ ]:
model.fit(X_train, [Y_train, Y_train2, Y_train3],
          validation_data = (X_test, [Y_test, Y_test2, Y_test3]), callbacks=[callbacks],
          batch_size = 64,
          epochs = 80,
          verbose = True,
          shuffle = False)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs

In [ ]:
# model.save("../A-G-E.h5")

In [ ]:
pred = model.predict(X_test)[0]
y_pred = [np.argmax(p) for p in pred]
y_true = [np.argmax(p) for p in Y_test] #age

print(classification_report(y_true, y_pred))

In [ ]:
pred = model.predict(X_test)[0]
y_pred = [np.argmax(p) for p in pred]
y_true = [np.argmax(p) for p in Y_test2] #gender

print(classification_report(y_true, y_pred))

In [ ]:
pred = model.predict(X_test)[0]
y_pred = [np.argmax(p) for p in pred]
y_true = [np.argmax(p) for p in Y_test3] #expression

print(classification_report(y_true, y_pred))